<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai tiktoken wandb tenacity weave

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb

In [3]:
# Download files on colab
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/{examples,prompt_template,system_template}.txt

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [5]:
import weave
from openai import OpenAI

wandb.init(project="llmapps", job_type="generation")
weave.init("llmapps")
client = OpenAI()

wandb: Currently logged in as: nick99 (itprodirect) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Initializing weave.
weave: Logged in as Weights & Biases user: nick99.
weave: View Weave data at https://wandb.ai/itprodirect/llmapps/weave


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [7]:
MODEL_NAME = "gpt-4o-mini"
# MODEL_NAME = "gpt-4"

In [8]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9ae3-71f9-71cf-adfc-7fec0f69e456


Sure! Here's a support question that a user of Weights & Biases (W&B) might ask:                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Support Question:                                                                                                  

Subject: Issue with Logging Custom Metrics in W&B                                                                  

Hi W&B Support Team,                                                                                               

I'm currently working on a machine learning project and I'm trying to log custom metrics using the W&B library.    
However, I'm facing an issue where the metrics don't seem to be appearing in the W&B dashboard as expected.        

I am using the following code snippet to log my metrics within the training loop:                                  

                                                                                                                   
 import wandb                                                                                                      
                                                                                                                   
 # Initialize a new W&B run                                                                                        
 wandb.init(project="my_project")                                                                                  
                                                                                                                   
 for epoch in range(num_epochs):                                                                                   
     # Training code...                                                                                            
     train_loss = compute_train_loss()                                                                             
     wandb.log({"train_loss": train_loss, "epoch": epoch})                                                         
                                                                                                                   
     # Validation code...                                                                                          
     val_loss = compute_val_loss()                                                                                 
     wandb.log({"val_loss": val_loss})                                                                             
                                                                                                                   

Despite this, I don't see the train_loss or val_loss in my dashboard. I've checked my internet connection and the  
project name is correct.                                                                                           

Could you please help me troubleshoot this issue? Are there any common mistakes I might be making?                 

Thank you!                                                                                                         

Best, [Your Name]                                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Feel free to customize the specifics of the question if needed!

Sure! Here’s a support question that a W&B (Weights & Biases) user might have:                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Subject: Issues with Syncing Local Runs to Dashboard                                                               

Hi W&B Support Team,                                                                                               

I'm currently using Weights & Biases to track my model experiments, but I've encountered an issue where my local   
runs are not syncing properly to the W&B dashboard. I've made sure to call wandb.init() at the beginning of my     
scripts, but the runs are still not appearing on my project dashboard.                                             

Here are some details about my setup:                                                                              

 • W&B version: 0.x.x (please confirm the current version)                                                         
 • Python version: 3.x.x                                                                                           
 • I’m running my scripts in a local environment (not in a cloud platform).                                        
 • Internet connection is stable.                                                                                  

I've tried restarting my session and even manually syncing using wandb sync, but it hasn’t fixed the issue.        

Could you please help me troubleshoot this?                                                                        

Thank you!                                                                                                         

Best, [Your Name]                                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Let me know if you need any more examples!

Sure! Here’s a support question that a WandB (Weights & Biases) user might ask:                                    

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Subject: Issue with Syncing Experiments to Dashboard                                                               

Message: Hi W&B Support Team,                                                                                      

I’m currently using Weights & Biases to track my experiments, but I’m facing an issue with syncing my latest runs  
to the dashboard. After running my training script, the dashboard does not show the latest metrics or the new      
experiments I initiated.                                                                                           

I've checked the API key and confirmed that it's set up correctly. I've also ensured that I'm calling wandb.init() 
at the beginning of my script. Is there anything else I can do to troubleshoot this issue?                         

Thanks for your help!                                                                                              

Best,                                                                                                              
[User's Name]                                                                                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Feel free to let me know if you need anything else!

Sure! Here's a potential support question from a Weights & Biases (W&B) user:                                      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Subject: Issue with Logging Custom Metrics in W&B                                                                  

Hi W&B Support Team,                                                                                               

I'm currently working on a machine learning project and I'm trying to log some custom metrics to my W&B dashboard. 
However, I'm encountering an issue where the metrics are not appearing as expected.                                

I am using the wandb.log() function to log metrics in my training loop, but the values seem to be missed or not    
updated in real-time. Here’s a snippet of the code I'm using:                                                      

                                                                                                                   
 import wandb                                                                                                      
                                                                                                                   
 wandb.init(project="my_project")                                                                                  
                                                                                                                   
 for epoch in range(10):                                                                                           
     # Simulate some training process                                                                              
     loss = train()  # Imagine this function returns the training loss                                             
     accuracy = evaluate()  # Imagine this function returns the accuracy                                           
                                                                                                                   
     # Logging metrics                                                                                             
     wandb.log({'loss': loss, 'accuracy': accuracy})                                                               
                                                                                                                   

I've also checked that I have the latest version of W&B installed. Can you help me troubleshoot what might be going
wrong? Are there any best practices for logging metrics that I should be aware of?                                 

Thank you!                                                                                                         

Best, [Your Name]                                                                                                  

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Let me know if you need anything else!

Sure! Here's a sample support question from a Weights & Biases (W&B) user:                                         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Subject: Issue with Experiment Logging in W&B                                                                      

Message:                                                                                                           

Hi W&B Support Team,                                                                                               

I hope this message finds you well. I'm currently running experiments using your platform, and I've encountered an 
issue while trying to log metrics for my model training.                                                           

I’m using the following code to log my metrics:                                                                    

                                                                                                                   
 import wandb                                                                                                      
                                                                                                                   
 wandb.init(project="my_project")                                                                                  
                                                                                                                   
 for epoch in range(num_epochs):                                                                                   
     # Training code here...                                                                                       
                                                                                                                   
     wandb.log({"epoch": epoch, "loss": loss, "accuracy": accuracy})                                               
                                                                                                                   

However, I've noticed that the metrics don’t seem to update in the W&B dashboard as expected. Sometimes they show  
up, but other times they don’t appear at all.                                                                      

Additionally, I'm unsure if I'm missing any configurations in my wandb.init() call.                                

Could you help me troubleshoot this issue? Is there any best practice I should follow to ensure that my metrics are
logged correctly?                                                                                                  

Thank you!                                                                                                         

Best regards,                                                                                                      
[Your Name]                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Feel free to modify it as needed!

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [9]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [11]:
delimiter = "\t"  # tab separated queries

# Try UTF-8 first (most likely), fallback if there are weird bytes
try:
    with open("examples.txt", "r", encoding="utf-8") as file:
        data = file.read()
except UnicodeDecodeError:
    with open("examples.txt", "r", encoding="utf-8", errors="replace") as file:
        data = file.read()

real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")
Markdown(f"Sample one:\n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "For distributed parallel runs, how can I get the run_id when I'm on a process other than RANK 0?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [12]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries:               
どんなことができますか？ why when i am running a WandbTracer and finish the run i can see the logs with the link   
from my colab, but when I try to check at my wandb profile there is not a single log there I Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [13]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9ae9-2fd8-7022-ac12-3ac14d36c28f


I'm trying to log some custom metrics in W&B, but they don't seem to appear in my dashboard. What might be going   
wrong?

I am trying to use W&B with my Jupyter notebook, but I'm having trouble logging my hyperparameters. Can you help me
figure out how to properly log my parameters so they show up in my W&B dashboard?

How can I link my Colab notebook to my W&B profile so that my experiment logs show up correctly?

Sure! Here’s a support question from a W&B user:                                                                   

"I'm trying to integrate W&B into my project, but I'm having trouble understanding how to log my custom metrics.   
Can someone explain how to log metrics manually, and are there specific functions I should use?"

How can I retrieve and visualize my W&B logs from a previous run in Jupyter Notebook?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [14]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

In [15]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [16]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[362, 2611, 2950, 4181, 809, 1210, 538, 955, 2105, 2528, 1666]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [17]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [18]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [19]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B     
                        documentation. Below you will find a fragment of W&B documentation:                        


               description: Collaborate and share W&B Reports with peers, co-workers, and your team.               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                             Collaborate on reports                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


Once you have saved a report, you can select the Share button to collaborate. A draft copy of the report is created
when you select the Edit button. Draft reports auto-save. Select Save to report to publish your changes to the     
shared report.                                                                                                     

A warning notification will appear if an edit conflict occurs. This can occur if you and another collaborator edit 
the same report at the same time. The warning notification will guide you to resolve potential edit conflicts.     

🌆 ]8;id=85778;@site/static/images/reports/share-report.gif\Report sharing modal for a report in a 'Public' project]8;;\                                                                                                                    

                                                Comment on reports                                                 

Click the comment button on a panel in a report to add a comment directly to that panel.                           

🌆 ]8;id=893390;/images/reports/demo_comment_on_panels_in_reports.gif\Adding a comment to a panel]8;;\                                                                                                                    

                                          Who can edit and share reports?                                          

Reports that are created within an individual's private project is only visible to that user. The user can share   
their project to a team or to the public.                                                                          

On team projects, both the administrator, or member who created the report, can toggle permissions between edit or 
view access for other team members. Team members can share reports.                                                

To share a report, select the Share button on the upper right hand corner.  You can either provide an email account
or copy the magic link. Users invited by email will need to log into Weights & Biases to view the report. Users who
are given a magic link to not need to log into Weights & Biases to view the report.                                

Shared reports are view-only.                                                                                      

Let's start!

Let's generate 3 possible questions:

In [ ]:
generate_and_print(system_prompt, generation_prompt, n=3)

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [33]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4o-mini"

In [34]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [35]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [36]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [37]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [38]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [39]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
is there any wandb utility function available to name a group randomly passed in wandb.init() similar to how wandb 
generates a random name for a single run (i.e. name argument in wandb.init()) can I make a project publicly        
viewable? wandb.errors.CommError: Sweep user not valid                                                             

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

               description: Collaborate and share W&B Reports with peers, co-workers, and your team.               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                             Collaborate on reports                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


Once you have saved a report, you can select the Share button to collaborate. A draft copy of the report is created
when you select the Edit button. Draft reports auto-save. Select Save to report to publish your changes to the     
shared report.                                                                                                     

A warning notification will appear if an edit conflict occurs. This can occur if you and another collaborator edit 
the same report at the same time. The warning notification will guide you to resolve potential edit conflicts.     

🌆 ]8;id=190906;@site/static/images/reports/share-report.gif\Report sharing modal for a report in a 'Public' project]8;;\                                                                                                                    

                                                Comment on reports                                                 

Click the comment button on a panel in a report to add a comment directly to that panel.                           

🌆 ]8;id=711654;/images/reports/demo_comment_on_panels_in_reports.gif\Adding a comment to a panel]8;;\                                                                                                                    

                                          Who can edit and share reports?                                          

Reports that are created within an individual's private project is only visible to that user. The user can share   
their project to a team or to the public.                                                                          

On team projects, both the administrator, or member who created the report, can toggle permissions between edit or 
view access for other team members. Team members can share reports.                                                

To share a report, select the Share button on the upper right hand corner.  You can either provide an email account
or copy the magic link. Users invited by email will need to log into Weights & Biases to view the report. Users who
are given a magic link to not need to log into Weights & Biases to view the report.                                

Shared reports are view-only.                                                                                      

───────────────────────────────────────

In [40]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [41]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [42]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-4592-7589-ac1f-a6ca15eb6259


('The user is collaborating with their team on a W&B report and wants to ensure that their colleagues can access and edit the report without issues. They are unsure about the sharing capabilities and the potential for edit conflicts when multiple people are working on the same document. The user has saved a draft of the report and is looking for clarification on how to manage sharing and permissions.\n',
 'How do I make sure my team can edit the report I created without running into conflicts? Also, can I share it easily with them?\n',
 'To ensure your team can edit the report, you should create the report within a team project. You can toggle permissions between edit or view access for team members as the administrator or report creator. Use the **Share** button on the upper right corner to share the report. You can provide an email account or share a magic link, which doesn’t require recipients to log into W&B to view the report. Be aware that if you and another collaborator edit the

In [43]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-6d2c-7f38-8441-892e9ad8723f
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-8176-70f8-9ec8-84033a28775b
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-9aec-737c-abd5-693792aa7d0d
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-b2ca-7b1e-a663-f9783516b597
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-d040-7e34-9874-b2f416f5ec51
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-e539-77be-a711-6e2935981339
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af0-fb1a-77dc-adf0-8cb7b451c8c6
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af1-1766-7782-a966-4be879434ecc
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af1-2e49-77a3-9740-aabcab8d1055
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af1-3f67-7694-af36-cf808155628f
weave: 🍩 https://wandb.ai/itprodirect/llmapps/r/call/019b9af1-60a0-7e48-bb24-1055fb1c55b3


<Artifact generated_examples>

In [44]:
wandb.finish()